In [ ]:
!pip install google-api-python-client
!pip install google_auth_oauthlib
!pip install beautifulsoup4

In [ ]:
# import the required libraries 
from googleapiclient.discovery import build 
from google_auth_oauthlib.flow import InstalledAppFlow 
from google.auth.transport.requests import Request 
import pickle
import base64 
import email 
from bs4 import BeautifulSoup 
import datetime

import os
import json
from dotenv import load_dotenv

from semantic_kernel.functions import kernel_function, KernelFunction
from typing import Annotated
import datetime

In [ ]:
class EmailPlugin:
    """Plugin for getting a list of emails."""
    def __init__(self):
        print("Initializing EmailPlugin.")
        # Define the access scopes. If modifying it, delete the token.pickle file. 
        self.scopes = ['https://www.googleapis.com/auth/gmail.readonly']
        self.emails = []

    def getDemoEmails(self):
        # Read demo data JSON file
        try:
            print("Reading demo data file.")
            with open("C:\\Repos\\AICourse\\PersonalFinanceAgent\\plugins\\email_plugin\\example_demo_data.json", 'r', encoding='utf-8') as f:
                emails = json.load(f)
                print("Demo data file read successfully.")
                return emails
        except Exception as e:
            print("Error reading demo data file: ", e)
        return []

    # Define the method to get the emails from the Gmail API
    def getRemoteEmails(self, start_date, end_date): 
        # Variable creds will store the user access token. 
        # If no valid token found, we will create one. 
        creds = None

        # The file token.pickle contains the user access token. 
        # Check if it exists 
        if os.path.exists('token.pickle'): 

            # Read the token from the file and store it in the variable creds 
            with open('token.pickle', 'rb') as token: 
                creds = pickle.load(token) 

        # If credentials are not available or are invalid, ask the user to log in. 
        if not creds or not creds.valid:
            flow = InstalledAppFlow.from_client_secrets_file('credentials.json', self.scopes) 
            creds = flow.run_local_server(port=0) 

            # Save the access token in token.pickle file for the next run 
            with open('token.pickle', 'wb') as token: 
                pickle.dump(creds, token) 

        # Connect to the Gmail API 
        service = build('gmail', 'v1', credentials=creds)

        # Search query with date range and other query parameters
        query = f'after:{start_date} before:{end_date} -category:promotions'

        # Call the Gmail API
        results = service.users().messages().list(maxResults=5000, userId='me', q=query).execute()
        messages = results.get('messages')

        return messages
    
    def getEachEmail(self, messages):
        # Variable creds will store the user access token. 
        # If no valid token found, we will create one. 
        creds = None

        # The file token.pickle contains the user access token. 
        # Check if it exists 
        if os.path.exists('token.pickle'): 
            # Read the token from the file and store it in the variable creds 
            with open('token.pickle', 'rb') as token: 
                creds = pickle.load(token) 

        # If credentials are not available or are invalid, ask the user to log in. 
        if not creds or not creds.valid:
            flow = InstalledAppFlow.from_client_secrets_file('credentials.json', self.scopes) 
            creds = flow.run_local_server(port=0) 

            # Save the access token in token.pickle file for the next run 
            with open('token.pickle', 'wb') as token: 
                pickle.dump(creds, token) 

        # Connect to the Gmail API 
        service = build('gmail', 'v1', credentials=creds)

        emails = []
        counter = 1

        # iterate through all the messages 
        for msg in messages: 
            # Get the message from its id 
            txt = service.users().messages().get(userId='me', id=msg['id']).execute() 

            # Use try-except to avoid any Errors 
            try: 
                # Get value of 'payload' from dictionary 'txt' 
                payload = txt['payload'] 
                headers = payload['headers'] 

                # Look for Subject and Sender Email in the headers 
                for d in headers: 
                    if d['name'] == 'Subject': 
                        subject = d['value'] 
                    if d['name'] == 'From': 
                        sender = d['value']
                    if d['name'] == 'Date':
                        date = d['value']

                # The Body of the message is in Encrypted format. So, we have to decode it. 
                # Get the data and decode it with base 64 decoder. 
                parts = payload.get('parts')[0] 
                data = parts['body']['data'] 
                data = data.replace("-","+").replace("_","/") 
                decoded_data = base64.b64decode(data)
                # Convert bytes to string
                decoded_string = decoded_data.decode('utf-8')

                email = dict()
                email['subject'] = subject
                email['sender'] = sender
                email['date'] = date
                email['body'] = decoded_string
                email['id'] = counter
                emails.append(email)

                counter += 1
            except:
                print("Error")

        return emails
        
    @kernel_function(name= "get_user_emails", description="Provides a list of emails.")
    def get_user_emails(self) -> Annotated[list, "Returns a list of emails."]:
        execution_mode = os.environ.get("EXECUTION_MODE")
        if execution_mode == "local":
            self.emails = self.getDemoEmails()
        else:
            # Define the date range (currently set to 15 days as single batch read call can fetch max 500 emails)
            # TODO: Add user input capability for date range and iteratively fetch emails
            start_date = '2025/03/01'
            end_date = '2025/03/15'
            messages = self.getRemoteEmails(start_date, end_date)
            self.emails = self.getEachEmail(messages)
        return self.emails